This notebook uses our models, trained on the dataset and saves the predictions for sea temperature 2021-2024 in a Parquet file

In [23]:
from matplotlib import pyplot as plt
from pandas import read_parquet 
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import pandas as pd 
import numpy as np 
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import TransformerModel, ExponentialSmoothing
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf

from darts.models import NBEATSModel
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller
from darts.metrics import mape, r2_score
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm

from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, SunspotsDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries

import warnings

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)
import os
pd.set_option('display.max_rows', 500)
pd.set_option("display.max_columns", 100)

RES_PATH = '../../res/'
MODELS_PATH = '../../'

In [25]:
# create list of model_names for loading pickle files
model_list = [f'model_{i}.pkl' for i in range(1000, 16000,1000)]
model_list = ['model_0.pkl'] + model_list
model_list

['model_0.pkl',
 'model_1000.pkl',
 'model_2000.pkl',
 'model_3000.pkl',
 'model_4000.pkl',
 'model_5000.pkl',
 'model_6000.pkl',
 'model_7000.pkl',
 'model_8000.pkl',
 'model_9000.pkl',
 'model_10000.pkl',
 'model_11000.pkl',
 'model_12000.pkl',
 'model_13000.pkl',
 'model_14000.pkl',
 'model_15000.pkl']

In [36]:
# visualize clean data
df = pd.read_parquet(RES_PATH + '/df_pixel.parquet')
df = df.set_index('date')
df.head()

,dd55xcg,dd55zb7,dd55zyg,dd57673,dd5775k,dd577gq,dd5789v,dd5793z,dd57b8m,dd57bwv,dd57c2r,dd57cqz,dd57d3c,dd57e1u,dd57ecy,dd57f23,dd57fqc,dd57g0k,dd57gbq,dd57gnu,dd57gyy,dd57kg2,dd57me6,dd57nku,dd57phy,dd57q7k,dd57r5q,dd57scb,dd57t9f,dd57ub2,dd57uyb,dd57v86,dd57vwf,dd57w3u,dd57x1y,dd57y2k,dd57yqu,dd57z0q,dd57zny,dd5dfqr,dd5dgq3,dd5dun7,dd5duyq,dd5dvy2,dd5dwxd,dd5dxrs,dd5dyw6,dd5dzqk,dd5e0hc,dd5e0ug,...,dehth48,dehw0er,dehw1e3,dehw29z,dehw39c,dehw47k,dehw63u,dehw88r,dehw8xp,dehw983,dehw9x1,dehwbtx,dehwct9,dehx0sp,dehx2dx,dehx89p,dek00ku,dek01hy,dek027k,dek035q,dek04hb,dek04uf,dek05sv,dek0652,dek06g6,dek07em,dek083u,dek091y,dek0b2k,dek0c0q,dek0d1b,dek0dcf,dek0e9v,dek0f02,dek0fb6,dek0hkz,dek0jkc,dek0k7r,dek0m73,dek0nhg,dek0nuv,dek0psz,dek0q57,dek0qgm,dek0rer,dek0s3z,dek0t3c,dek20sf,dek21ku,dek24hy
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-01-01,26.693710,26.681581,26.681581,26.697742,26.697742,26.705161,26.695484,26.695484,26.681581,26.681581,26.681581,26.681581,26.697742,26.697742,26.705161,26.677355,26.677355,26.677355,26.680968,26.677355,26.680968,26.705161,26.705161,26.714387,26.714387,26.714387,26.714387,26.705161,26.705161,26.680968,26.680968,26.680968,26.680968,26.714387,26.714387,26.687710,26.687710,26.687710,26.687710,26.708581,26.715742,26.715742,26.715742,26.723032,26.723032,26.735355,26.723032,26.735355,26.716903,26.716903,...,25.905226,25.927581,25.927581,25.961129,25.961129,25.922290,25.961645,25.961129,25.975129,25.961129,25.975129,25.975129,25.975129,25.983323,25.983323,25.983323,25.945839,25.945839,25.945839,25.945839,25.967000,25.967000,25.965419,25.967000,25.967000,25.965419,25.925323,25.925323,25.925323,25.925323,25.937000,25.937000,25.942129,25.937000,25.937000,25.965419,25.965419,25.965419,25.965419,25.968710,25.968710,25.964258,25.968710,25.968710,25.964258,25.942129,25.942129,25.964258,25.972548,25.972548
2000-02-01,25.859448,25.944000,25.944000,26.149759,26.149759,26.252552,25.964828,25.964828,26.083759,26.083759,26.083759,26.083759,26.149759,26.149759,26.252552,26.241000,26.241000,26.241000,26.289828,26.241000,26.289828,26.252552,26.252552,26.293483,26.293483,26.293483,26.293483,26.252552,26.252552,26.289828,26.289828,26.289828,26.289828,26.293483,26.293483,26.322966,26.322966,26.322966,26.322966,26.342379,26.349069,26.349069,26.349069,26.331034,26.331034,26.345931,26.331034,26.345931,26.344690,26.344690,...,25.581207,25.533793,25.533793,25.503103,25.503103,25.525690,25.493448,25.503103,25.481345,25.503103,25.481345,25.481345,25.481345,25.422138,25.422138,25.422138,25.452517,25.452517,25.452517,25.452517,25.427586,25.427586,25.395483,25.427586,25.427586,25.395483,25.474621,25.474621,25.474621,25.474621,25.459828,25.459828,25.431379,25.459828,25.459828,25.395483,25.395483,25.395483,25.395483,25.358448,25.358448,25.331931,25.358448,25.358448,25.331931,25.431379,25.431379,25.331931,25.314241,25.314241
2000-03-01,26.409710,26.386871,26.386871,26.402806,26.402806,26.351032,26.422097,26.422097,26.392419,26.392419,26.392419,26.392419,26.402806,26.402806,26.351032,26.368290,26.368290,26.368290,26.319000,26.368290,26.319000,26.351032,26.351032,26.293161,26.293161,26.293161,26.293161,26.351032,26.351032,26.319000,26.319000,26.319000,26.319000,26.293161,26.293161,26.266387,26.266387,26.266387,26.266387,26.182903,26.134968,26.134968,26.134968,26.154226,26.154226,26.164839,26.154226,26.164839,26.228323,26.228323,...,25.293677,25.341452,25.341452,25.345129,25.345129,25.296258,25.309226,25.345129,25.355129,25.345129,25.355129,25.355129,25.355129,25.352516,25.352516,25.352516,25.367613,25.367613,25.367613,25.367613,25.359774,25.359774,25.359645,25.359774,25.359774,25.359645,25.368065,25.368065,25.368065,25.368065,25.367645,25.367645,25.374355,25.367645,25.367645,25.359645,25.359645,25.359645,25.359645,25.351323,25.351323,25.350419,25.351323,25.351323,25.350419,25.374355,25.374355,25.350419,25.373452,25.373452
2000-04-01,26.154400,26.174433,26.174433,26.

In [48]:
def prepare_input(df, start_col=0, end_col=1000):
    # function loads subset of points from parquet and generates TimeSeries object
    transformer = Scaler() 
    # train: 2014-2020, valid: 2021
    train, val = df.iloc[161:245, start_col:end_col], df.iloc[245:,start_col:end_col]
    print((start_col, end_col))
   
    train = TimeSeries.from_dataframe(train, freq='MS')
    val = TimeSeries.from_dataframe(val, freq='MS')

    train_ts = transformer.fit_transform(train)
    val_ts = transformer.transform(val)
    year_series = datetime_attribute_timeseries(
        pd.date_range(start=train_ts.start_time(), freq=train_ts.freq_str, periods=120),
        attribute="year",
        one_hot=False,
    )
    year_series = Scaler().fit_transform(year_series)
    month_series = datetime_attribute_timeseries(
        year_series, attribute="month", one_hot=True
    )
    covariates = year_series.stack(month_series)
    cov_train, cov_val = covariates.split_after(pd.Timestamp("20201201"))

    return (train_ts, val_ts), (cov_train, cov_val), covariates

In [ ]:
import pickle 

pred_21_24 = None

# generate predictions up till 2024 for every subset of points and save to single dataframe
for m_path in model_list:
    print(m_path)
    start_idx = int(m_path.split('.')[0].split('_')[1])
    (train_ts, val_ts), (cov_train, cov_val), covariates = prepare_input(df, start_idx, start_idx+1000)

    model = pickle.load(open(f'../../{m_path}','rb'))

    pred_series = model.predict(n=36, future_covariates=covariates).values()
    pred_df = pd.DataFrame(pred_series, columns=df.columns[start_idx:start_idx+1000])


    if pred_21_24 is None:
        pred_21_24 = pred_df
    else:
        pred_21_24 = pd.concat([pred_21_24, pred_df], axis=1)



In [70]:
# re-join indices to predictions for output
datetime_idx = pd.date_range(start='20210101', end='20231201', freq='MS')

pred_21_24.reset_index() 
pred_21_24.index = datetime_idx

In [71]:
# verify output
pred_21_24.head()

,dd55xcg,dd55zb7,dd55zyg,dd57673,dd5775k,dd577gq,dd5789v,dd5793z,dd57b8m,dd57bwv,dd57c2r,dd57cqz,dd57d3c,dd57e1u,dd57ecy,dd57f23,dd57fqc,dd57g0k,dd57gbq,dd57gnu,dd57gyy,dd57kg2,dd57me6,dd57nku,dd57phy,dd57q7k,dd57r5q,dd57scb,dd57t9f,dd57ub2,dd57uyb,dd57v86,dd57vwf,dd57w3u,dd57x1y,dd57y2k,dd57yqu,dd57z0q,dd57zny,dd5dfqr,dd5dgq3,dd5dun7,dd5duyq,dd5dvy2,dd5dwxd,dd5dxrs,dd5dyw6,dd5dzqk,dd5e0hc,dd5e0ug,...,dehth48,dehw0er,dehw1e3,dehw29z,dehw39c,dehw47k,dehw63u,dehw88r,dehw8xp,dehw983,dehw9x1,dehwbtx,dehwct9,dehx0sp,dehx2dx,dehx89p,dek00ku,dek01hy,dek027k,dek035q,dek04hb,dek04uf,dek05sv,dek0652,dek06g6,dek07em,dek083u,dek091y,dek0b2k,dek0c0q,dek0d1b,dek0dcf,dek0e9v,dek0f02,dek0fb6,dek0hkz,dek0jkc,dek0k7r,dek0m73,dek0nhg,dek0nuv,dek0psz,dek0q57,dek0qgm,dek0rer,dek0s3z,dek0t3c,dek20sf,dek21ku,dek24hy
2021-01-01,0.277048,0.253203,0.273774,0.274629,0.271244,0.281410,0.261097,0.279309,0.260988,0.287099,0.255420,0.265942,0.267552,0.273500,0.277315,0.293452,0.288745,0.304619,0.319825,0.289169,0.278596,0.275672,0.267061,0.235924,0.232394,0.270915,0.269084,0.254543,0.295147,0.277358,0.307515,0.290780,0.294952,0.263930,0.270948,0.290806,0.294416,0.306400,0.281794,0.274639,0.292620,0.270570,0.262109,0.299457,0.277061,0.272921,0.285930,0.270362,0.286891,0.257054,...,0.076075,0.076871,0.061160,0.079903,0.073457,0.060189,0.071704,0.076142,0.077752,0.084145,0.081503,0.083870,0.078456,0.080006,0.093691,0.095531,0.089821,0.085802,0.093358,0.091926,0.091465,0.086291,0.095071,0.092566,0.090159,0.100562,0.083340,0.087308,0.087880,0.089827,0.083199,0.097407,0.081486,0.099686,0.103551,0.081693,0.076563,0.092996,0.097691,0.087581,0.089784,0.103068,0.085546,0.088318,0.094037,0.077473,0.084554,0.079240,0.099612,0.090954
2021-02-01,0.182109,0.163174,0.155246,0.182579,0.158009,0.183415,0.164119,0.172962,0.173457,0.184333,0.163813,0.172368,0.169995,0.159747,0.187090,0.183803,0.179761,0.193051,0.202333,0.183612,0.187185,0.182122,0.175563,0.156778,0.154759,0.183487,0.177814,0.169852,0.189461,0.174090,0.187012,0.178771,0.185496,0.166684,0.168263,0.182105,0.183716,0.194814,0.179524,0.185142,0.187990,0.180994,0.168971,0.172750,0.166678,0.165828,0.186395,0.165498,0.186151,0.153083,...,0.058584,0.036581,0.028251,0.023392,0.021683,0.017588,0.026369,0.022611,0.027553,0.032568,0.025077,0.034086,0.019599,0.023897,0.035142,0.025523,0.013237,0.024380,0.014858,0.012238,0.017513,0.016712,0.024006,0.020723,0.020205,0.023641,0.015404,0.016371,0.010406,0.014482,0.018622,0.018998,0.015754,0.027592,0.030055,0.019103,0.017429,0.023312,0.022216,0.023135,0.015932,0.025985,0.025075,0.022519,0.021170,0.015875,0.009229,0.015026,0.019407,0.018096
2021-03-01,0.180092,0.168480,0.162386,0.192985,0.171293,0.177264,0.176778,0.179519,0.180117,0.182071,0.170214,0.173700,0.183019,0.171947,0.178138,0.192049,0.172941,0.190532,0.174118,0.177636,0.170018,0.179822,0.175405,0.141741,0.152689,0.159030,0.159903,0.168593,0.172981,0.155646,0.175149,0.159681,0.163237,0.149387,0.157783,0.156701,0.155705,0.164524,0.153082,0.166515,0.154624,0.149500,0.150319,0.132804,0.123618,0.140969,0.154820,0.136142,0.149679,0.136848,...,0.017196,0.022325,0.024664,0.028357,0.029330,0.019606,0.034510,0.034188,0.035712,0.043701,0.039768,0.040442,0.038688,0.038214,0.034234,0.043609,0.040343,0.046810,0.041432,0.042613,0.043065,0.036189,0.043902,0.038277,0.051582,0.050962,0.038264,0.038564,0.041630,0.049515,0.041590,0.049442,0.038491,0.046929,0.047571,0.043872,0.049870,0.046903,0.044675,0.043954,0.041392,0.056208,0.049021,0.053005,0.055808,0.041942,0.042987,0.045011,0.051456,0.045760
2021-04-01,0.187316,0.183719,0.186032,0.203647,0.184816,0.189056,0.191893,0.194587,0.206795,0.192237,0.204510,0.195631,0.203087,0.197509,0.188195,0.213160,0.200170,0.226200,0.201115,0.190151,0.199489,0.197606,0.195839,0.183636,0.185271,0.185859,0.179169,0.192258,0.191294,0.199824,0.209284,0.186266,0.191497,0.181506,0.189387,0.209073,0.193593,0.200334,0.195475,0.211592,0.206438,0.192526,0.202308,0.189423,0.196822,0.214280,0.208152,0.204324,0.18415

In [72]:
# export output to parquet files
pred_21_24.to_parquet('pred_21_24.parquet')